In [11]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [12]:
#Config
TRAIN_PATH = 'Task_A/train'
VAL_PATH = 'Task_A/val'
BATCH_SIZE = 16
EPOCHS = 15
EMBEDDING_DIM = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [13]:
#MTCNN + FaceNet
mtcnn = MTCNN(image_size=160, margin=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
for param in resnet.parameters():
    param.requires_grad = False

In [14]:
#Binary Dataset
class GenderFaceDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        self.label_map = {'male': 0, 'female': 1}  

        for gender in ['male', 'female']:
            gender_dir = os.path.join(root_dir, gender)
            if not os.path.isdir(gender_dir):
                continue
            for fname in os.listdir(gender_dir):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    self.samples.append((os.path.join(gender_dir, fname), self.label_map[gender]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        face = mtcnn(img)
        if face is None:
            return self.__getitem__((idx + 1) % len(self.samples))
        with torch.no_grad():
            embedding = resnet(face.unsqueeze(0).to(device)).squeeze(0).cpu()
        return embedding, label


In [25]:
# Binary Classifier 
class BinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=512):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.classifier(x)

In [26]:
#Data Loaders
train_dataset = GenderFaceDataset(TRAIN_PATH)
val_dataset = GenderFaceDataset(VAL_PATH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [27]:
# Model, Loss, Optimizer 
model = BinaryClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [28]:
#  Early Stopping 
best_val_acc = 0
patience = 3
counter = 0

In [29]:
# Training Loop 
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    for embeddings, labels in tqdm(train_loader, desc='Training'):
        embeddings = embeddings.to(device)
        labels = labels.float().unsqueeze(1).to(device)  

        outputs = model(embeddings)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_loss = total_loss / len(train_loader)

    # ===== Validation =====
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for embeddings, labels in tqdm(val_loader, desc='Validation'):
            embeddings = embeddings.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(embeddings)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss /= len(val_loader)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.2f}%")

    # ===== Early Stopping =====
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        torch.save(model.state_dict(), 'best_binary_classifier.pth')
        print(f"Best model saved (Val Acc: {val_acc:.2f}%)")
    else:
        counter += 1
        print(f"No improvement for {counter} epoch(s)")

    if counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 1/15


Validation: 100%|██████████████████████████████████████████████████████████████████████| 27/27 [01:00<00:00,  2.24s/it]


Train Loss: 0.2592 | Train Acc: 89.30%
Val   Loss: 0.3600 | Val   Acc: 90.05%
Best model saved (Val Acc: 90.05%)

Epoch 2/15


Validation: 100%|██████████████████████████████████████████████████████████████████████| 27/27 [01:05<00:00,  2.41s/it]


Train Loss: 0.1484 | Train Acc: 94.81%
Val   Loss: 0.3076 | Val   Acc: 91.47%
Best model saved (Val Acc: 91.47%)

Epoch 3/15


...

Train Loss: 0.0741 | Train Acc: 97.56%
Val   Loss: 0.3908 | Val   Acc: 91.00%
No improvement for 1 epoch(s)

Epoch 7/15


Validation: 100%|██████████████████████████████████████████████████████████████████████| 27/27 [00:58<00:00,  2.18s/it]


Train Loss: 0.0642 | Train Acc: 97.66%
Val   Loss: 0.4061 | Val   Acc: 90.52%
No improvement for 2 epoch(s)

Epoch 8/15


Validation: 100%|██████████████████████████████████████████████████████████████████████| 27/27 [00:59<00:00,  2.19s/it]

Train Loss: 0.0689 | Train Acc: 97.51%
Val   Loss: 0.4037 | Val   Acc: 90.76%
No improvement for 3 epoch(s)
Early stopping triggered.


In [ ]:
from PIL import Image
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load face detector and embedding model
mtcnn = MTCNN(image_size=160, margin=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

for param in resnet.parameters():
    param.requires_grad = False

# Binary classifier model definition
class BinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=512):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.classifier(x)

# Load trained model
model = BinaryClassifier().to(device)
model.load_state_dict(torch.load('best_binary_classifier.pth', map_location=device))
model.eval()


In [ ]:
def predict_gender(image_path):
    img = Image.open(image_path).convert('RGB')

    # Detect and crop face
    face = mtcnn(img)
    if face is None:
        print("❌ No face detected.")
        return

    # Generate embedding
    with torch.no_grad():
        embedding = resnet(face.unsqueeze(0).to(device))
        output = model(embedding)
        prediction = (output > 0.5).float().item()

    label = "Female" if prediction == 1 else "Male"
    print(f"✅ Predicted Gender: {label}")


In [ ]:
predict_gender("Task_A/val/male/018_frontal.jpg")

In [ ]:
predict_gender("Task_A/val/female/Jennifer_Keller_0001.jpg")